In [1]:
import pandas as pd, numpy as np, pymc as pm

- pull in age x sex structure for a specific block

- pull in decennial hh sizes
- assign arbitrary hhids, relationships

- run mcmc

# Test out household structure assignment with mcmc

Objective: given a synthetic dataset that contains, for each individual, their census block, precise age, sex, and precise race, assign a realistic household structure (wrt household ids and relationships to head of household) using distributions from the 2010 decennial.

## pull in data

This is synthetic data; it was generated using the 2010 decennial and the 2018 acs (should switch to 2010)

It has fairly accurate age, sex, and race distribution for each census block. The relationship and ethnicity (hispanic binary) columns are bad, because they are informed only by the state-level distribution of each var from the ACS. We will scrap those relationship assigments and reassign them using MCMC and the block-level decennial data. The inaccuracy of the ethnicity still needs to be addressed.

In [2]:
## pull in age-sex structure for a single block
df = pd.read_csv('/home/j/temp/beatrixh/sim_science/outputs/example_hh_structure_assignment/ravenna_07_14_2020.csv')
df.head()

,state,county,tract,block,age,sex_id,relationship,hispanic,racaian,racasn,racblk,racnhpi,racsor,racwht,pweight,geoid
0,53,33,4301,3011,22,1,2,0,0,0,1,0,0,1,1,530330043013011
1,53,33,4301,3011,52,1,1,0,0,0,0,0,0,1,1,530330043013011
2,53,33,4301,3011,71,1,0,0,0,0,0,0,0,1,1,530330043013011
3,53,33,4301,3011,38,2,2,0,0,0,0,0,0,1,1,530330043013011
4,53,33,4301,3011,37,2,1,0,0,0,0,0,0,1,1,530330043013011


In [3]:
## pull in df to fit to from decennial
input_dir = '/home/j/temp/beatrixh/sim_science/decennial_census_2010/'
location_cols = ['STATE', 'COUNTY', 'TRACT', 'BLKGRP', 'BLOCK']
household_sizes = ['P02800' + ('00' + str(i))[-2:] for i in np.arange(1,17)]
relations_present = ['P02900' + ('00' + str(i))[-2:] for i in np.arange(1,29)]
household_type = ['P03000' + ('00' + str(i))[-2:] for i in np.arange(1,14)]

In [4]:
# this is the 2010 decennial for a single block in ravenna
ravenna = pd.read_csv('/home/j/temp/beatrixh/sim_science/outputs/example_hh_structure_assignment/decennial_ravenna_07_20_2020.csv')

## prep vars of interest

Decennial var labels from https://www2.census.gov/programs-surveys/decennial/2010/technical-documentation/complete-tech-docs/summary-file/sf1.pdf?#

In [5]:
#relationship to head of hh
relp_labels = ['total', 'in_households', 'in_family_households',
               'householders_family', 'male_householder_family', 
               'female_householder_family', 'spouse', 'biological_child',
               'adopted_child', 'stepchild', 'grandchild', 'sibling', 
               'parent', 'parent_in_law', 'child_in_law', 
               'other_relatives', 'nonrelatives', 
               'in_nonfamily_households', 
               'male_householder_nonfamily','male_householder_nonfamily_alone','male_householder_nonfamily_not_alone', 
               'female_householder_nonfamily', 'female_householder_nonfamily_alone','female_householder_nonfamily_not_alone', 
               'nonrelatives_nonfamily',
               'in_gq','gq_inst','gq_noninst']

ravenna_relps = ravenna[relations_present].melt(value_name = 'counts', var_name = 'relp')
ravenna_relps['decennial_relp'] = relp_labels

#these will be onto and mutually exclusive, but not 1:1 or defined on the entire domain
decennial_acs_relp_dict = {'householders_family':0,
                           'male_householder_nonfamily':0,
                           'female_householder_nonfamily':0,
                           'spouse':1,
                          'biological_child':2,
                          'adopted_child':3,
                          'stepchild':4,
                          'sibling':5,
                          'parent':6,
                          'grandchild':7,
                          'parent_in_law':8,
                          'child_in_law':9,
                          'other_relatives':10,
                          'nonrelatives':15, #refine this 
                          'nonrelatives_nonfamily':15, #refine this
                          'gq_inst':16,
                          'gq_noninst':17} #same-sex spouses were mapped to nonrelatives

ravenna_relps['relp_id'] = ravenna_relps.decennial_relp.map(decennial_acs_relp_dict)
ravenna_relps = ravenna_relps[ravenna_relps.relp_id.notna()]
ravenna_relps = ravenna_relps.drop(columns=['relp','decennial_relp'])
ravenna_relps = ravenna_relps.groupby('relp_id').sum().reset_index()

In [6]:
## household type
hh_type_labels = ['total','in_family_hh',
                  'in_husband_wife_family',
                  'in_other_family','in_male_head_no_wife',
                  'in_female_head_no_husband','in_non_family',
                  'non_family_male_head','non_family_male_head_alone',
                  'non_family_male_head_not_alone',
                  'non_family_female_head','non_family_female_head_alone',
                  'non_family_female_head_not_alone']

ravenna_hh_types = ravenna[household_type].melt(value_name = 'counts', var_name = 'hh_type')
ravenna_hh_types['hh_type_label'] = hh_type_labels

In [7]:
## household ids
ravenna_hhs = ravenna[household_sizes].melt(value_name = 'counts', var_name = 'hh_size')

## map census vars to household sizes
d = {}
d.update({i:int(i[-1])-1 for i in ravenna.columns[8:14]})
d.update({i:int(i[-1])+1 for i in ravenna.columns[15:]})

ravenna_hhs['hh_size'] = ravenna_hhs.hh_size.map(d)
ravenna_hhs = ravenna_hhs[ravenna_hhs.hh_size.notna()]
ravenna_hhs = ravenna_hhs.groupby('hh_size').sum().reset_index()

In [8]:
n_households = ravenna_hhs['counts'].sum()
n_sims = df.pweight.sum()

print(f'''n_households: {n_households}
n_simulants: {n_sims}''')

n_households: 24
n_simulants: 65


## mcmc

initialize with uniform priors, then add in constraints using decennial block-level data

In [9]:
hhid = pm.DiscreteUniform('hhid', 0, n_households-1, size=n_sims)
relp = pm.DiscreteUniform('relp', 0, ravenna_relps.relp_id.max(), size = n_sims)
family_hh = pm.DiscreteUniform('family_hh', 0, 1, size = n_sims)

In [10]:
#make sure there are exactly n_households hhs
@pm.potential
def hh_count_constraint_soft(hhid=hhid, n_households=n_households):
    diff = len(np.unique(hhid)) - n_households
    return -1*(diff)**2

In [11]:
def checkout_hh_count_constraint_soft(hhid=hhid.value, n_households=n_households):
    diff = len(np.unique(hhid)) - n_households
    print(f'difference in number of unique households: {diff}')

In [12]:
#constrain towards correct houshold sizes
@pm.potential
def hh_size_distribution_constraint_soft(hhid=hhid, target=ravenna_hhs):
    #count hh_size distribution in sim data
    sim_hh_sizes = pd.DataFrame({'hhids':hhid}).hhids.value_counts().value_counts()
    sim_hh_sizes = pd.DataFrame(sim_hh_sizes).reset_index().rename(columns={'index':'hh_size','hhids':'sim_counts'})
    
    #compare to target distribution
    compare = sim_hh_sizes.merge(target, on = 'hh_size', how = 'outer').fillna(0)
    compare['abs_diff'] = np.abs(compare.sim_counts - compare.counts)
    
    return -compare.abs_diff.sum()

In [13]:
def checkout_hh_size_distribution_constraint_soft(hhid=hhid.value, target=ravenna_hhs):
    #count hh_size distribution in sim data
    sim_hh_sizes = pd.DataFrame({'hhids':hhid}).hhids.value_counts().value_counts()
    sim_hh_sizes = pd.DataFrame(sim_hh_sizes).reset_index().rename(columns={'index':'hh_size','hhids':'sim_counts'})
    
    #compare to target distribution
    compare = sim_hh_sizes.merge(target, on = 'hh_size', how = 'outer').fillna(0)
    compare['abs_diff'] = np.abs(compare.sim_counts - compare.counts)
    
    return compare.sort_values('hh_size')

In [14]:
#constrain towards correct relationship assignment distribution
@pm.potential
def relp_distribution_constraint_soft(relp=relp, target = ravenna_relps):
    #count relp distribution in sim data
    sim_relps = pd.DataFrame({'relps':relp}).relps.value_counts()
    sim_relps = pd.DataFrame(sim_relps).reset_index().rename(columns={'index':'relp_id','relps':'sim_counts'})

    #compare to target distribution
    compare = sim_relps.merge(ravenna_relps, on='relp_id', how = 'outer').fillna(0)
    compare['abs_diff'] = np.abs(compare.sim_counts - compare.counts)
    
    return -compare.abs_diff.sum()

In [15]:
def checkout_relp_distribution_constraint_soft(relp=relp.value, target = ravenna_relps):
    #count relp distribution in sim data
    sim_relps = pd.DataFrame({'relps':relp}).relps.value_counts()
    sim_relps = pd.DataFrame(sim_relps).reset_index().rename(columns={'index':'relp_id','relps':'sim_counts'})

    #compare to target distribution
    compare = sim_relps.merge(ravenna_relps, on='relp_id', how = 'outer').fillna(0)
    compare['abs_diff'] = np.abs(compare.sim_counts - compare.counts)
    
    return compare.sort_values('relp_id')

In [16]:
#make sure there are exactly n_households hh_heads
@pm.potential
def hh_head_count_constraint_soft(relp=relp, n_households=n_households):
    diff = sum([i==0 for i in relp]) - n_households
    return -1*(diff)**2

In [17]:
def checkout_hh_head_count_constraint_soft(relp=relp.value, n_households=n_households):
    diff = sum([i==0 for i in relp]) - n_households
    print(f'difference in number of household heads: {diff}')

In [18]:
#make sure one hh_head per hhid
@pm.potential
def one_head_per_hh_soft_constraint(relp=relp, hhid=hhid):
    #count hh_heads
    hh_heads = [1 if i==0 else 0 for i in relp]
    
    #enumerate per hhid
    heads_per_hh = pd.DataFrame({'hhid':hhid,'hh_head':hh_heads})
    heads_per_hh = heads_per_hh.groupby('hhid').sum()
    
    #look at how far from 1 per hh each is
    heads_per_hh.hh_head -= 1
    
    return -sum(heads_per_hh.hh_head.abs())

In [19]:
def checkout_one_head_per_hh_soft_constraint(relp=relp.value, hhid=hhid.value):
    #count hh_heads
    hh_heads = [1 if i==0 else 0 for i in relp]
    
    #enumerate per hhid
    heads_per_hh = pd.DataFrame({'hhid':hhid,'hh_head_count':hh_heads})
    heads_per_hh = heads_per_hh.groupby('hhid').sum()
    
    return heads_per_hh.sort_values('hhid')

In [20]:
#make sure there are correct count of family hhs
@pm.potential
def family_hh_count_soft_constraint(family_hh=family_hh, target_hh_types=ravenna_hh_types):
    diff = sum(family_hh) - target_hh_types[target_hh_types.hh_type_label=='in_family_hh'].counts
    return -1*(np.abs(diff))

In [21]:
def checkout_family_hh_count_soft_constraint(family_hh=family_hh.value, target_hh_types=ravenna_hh_types):
    diff = sum(family_hh) - target_hh_types[target_hh_types.hh_type_label=='in_family_hh'].counts
    print(f'difference in count of actual family households vs. synthetic family households: {diff.values[0]}')

In [22]:
#make sure there leq 1 spouse per family hh, and 0 spouse per non family hh
@pm.potential
def assign_spouses_correctly_soft_constraint(relp=relp, hhid=hhid, family_hh=family_hh):
    # pull all vars
    df = pd.DataFrame({'relp':relp, 'hhid': hhid, 'family_hh':family_hh})
    fam = df[df.family_hh==1]
    non_fam = df[df.family_hh==0]
    
    #enumerate extra spouses in family hhs with >1 spouse
    fam.loc[:,'spouses_count'] = [1 if i==1 else 0 for i in fam.relp]
    fam = fam.groupby('hhid').sum()
    extra_spouses_fam = sum([i-1 if i>1 else 0 for i in fam.spouses_count])
    
    #enumerate extra spouses in non family hhs (should this be penalized more?)
    extra_spouses_nonfam = sum([1 if i==1 else 0 for i in non_fam.relp])
    
    return -0.5*(extra_spouses_fam + extra_spouses_nonfam)

/share/code/beatrixh/miniconda/envs/mcmc/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/share/code/beatrixh/miniconda/envs/mcmc/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [23]:
def checkout_assign_spouses_correctly_soft_constraint(relp=relp.value, hhid=hhid.value, family_hh=family_hh.value):
    # pull all vars
    df = pd.DataFrame({'relp':relp, 'hhid': hhid, 'family_hh':family_hh})
    fam = df[df.family_hh==1]
    non_fam = df[df.family_hh==0]
    
    #enumerate extra spouses in family hhs with >1 spouse
    fam.loc[:,'spouses_count'] = [1 if i==1 else 0 for i in fam.relp]
    fam = fam.groupby('hhid').sum()
    extra_spouses_fam = sum([i-1 if i>1 else 0 for i in fam.spouses_count])
    
    #enumerate extra spouses in non family hhs (should this be penalized more?)
    extra_spouses_nonfam = sum([1 if i==1 else 0 for i in non_fam.relp])
    
    print(f'''    extra spouses in family households: {extra_spouses_fam}
    extra spouses in non-fam households: {extra_spouses_nonfam}''')

In [24]:
m = pm.Model([hhid, relp])
mc = pm.MCMC(m)

/share/code/beatrixh/miniconda/envs/mcmc/lib/python3.6/site-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


In [25]:
mc.sample(iter=10_000, burn = 1_000)

 [-----------------100%-----------------] 10000 of 10000 complete in 128.2 sec

In [33]:
mc.sample(iter=10_000)

 [-----------------100%-----------------] 10000 of 10000 complete in 201.0 sec

## checkout fit

For each var we fit on, compare the counts for each value of that var in the simulated data vs. the decennial data

In [34]:
checkout_hh_count_constraint_soft(hhid=hhid.value, n_households=n_households)

difference in number of unique households: -1


In [35]:
checkout_hh_size_distribution_constraint_soft(hhid=hhid.value, target=ravenna_hhs)

,hh_size,sim_counts,counts,abs_diff
1,1,5.0,6,1.0
0,2,8.0,9,1.0
4,3,2.0,1,1.0
3,4,3.0,3,0.0
2,5,4.0,4,0.0
5,6,1.0,1,0.0
6,7,0.0,0,0.0


In [36]:
checkout_relp_distribution_constraint_soft(relp=relp.value, target = ravenna_relps)

,relp_id,sim_counts,counts,abs_diff
0,0,24.0,24.0,0.0
3,1,6.0,12.0,6.0
2,2,6.0,7.0,1.0
8,3,2.0,2.0,0.0
14,4,1.0,0.0,1.0
15,5,0.0,0.0,0.0
13,6,1.0,0.0,1.0
16,7,0.0,0.0,0.0
5,8,3.0,2.0,1.0
12,9,1.0,0.0,1.0


In [47]:
checkout_hh_head_count_constraint_soft(relp=relp.value, n_households=n_households)

difference in number of household heads: 0


In [48]:
checkout_one_head_per_hh_soft_constraint(relp=relp.value, hhid=hhid.value)

,hh_head_count
hhid,
0,2
1,1
2,1
3,0
4,1
6,2
7,1
8,1
9,1


In [49]:
checkout_family_hh_count_soft_constraint(family_hh=family_hh.value, target_hh_types=ravenna_hh_types)

difference in count of actual family households vs. synthetic family households: -7


In [50]:
checkout_assign_spouses_correctly_soft_constraint(relp=relp.value, hhid=hhid.value, family_hh=family_hh.value)

    extra spouses in family households: 0
    extra spouses in non-fam households: 4


/share/code/beatrixh/miniconda/envs/mcmc/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/share/code/beatrixh/miniconda/envs/mcmc/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
